<center><h1>OCR Google Vision API</h1></center>

<h2>Introduction</h2>

In [ ]:
!pip install google-cloud
!pip install google-cloud-pubsub
!pip install google-cloud-translate

In [ ]:
from pdf2image import convert_from_path
import glob
from tqdm import tqdm
import base64
import json
import os

from google.cloud import pubsub_v1
from google.cloud import storage
from google.cloud import translate_v2 as translate
from google.cloud import vision

from google.oauth2 import service_account
import googleapiclient.discovery

tqdm().pandas()

<h2>I - Convert PDF into JPEG</h2>

In [ ]:
list_ = glob.glob("extract/*.pdf")
#list_= ["extract/22453181.pdf"]
for i in list_[:1]:
    print(i)
    pages = convert_from_path(i, 500)
    
    for page in tqdm(enumerate(pages)):
        
        page[1].save("test_pdf_ocr/"+i.split('/')[1].split('.')[0]+'_'+str(page[0])+'.jpg', 'JPEG')

<h2>II - Set up credentials</h2>

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/cloud-vision']
SERVICE_ACCOUNT_FILE = "/opt/continuum/project/config/prod-datascience-dsp-267215-67e4faebee83.json"

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [ ]:
vision = googleapiclient.discovery.build('vision', 'v1', credentials=credentials)
translate_client = translate.Client()
publisher = pubsub_v1.PublisherClient()
storage_client = storage.Client()

<h2>III - API Vision request</h2>

In [ ]:
def detect_text(path, credentials):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient(credentials=credentials)

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    annotations = response.text_annotations
    if len(annotations) > 0:
        text = annotations[0].description
    else:
        text = ''
    print('Extracted text {} from image ({} chars).'.format(text, len(text)))

    detect_language_response = translate_client.detect_language(text)
    src_lang = detect_language_response['language']
    print('Detected language {} for text {}.'.format(src_lang, text))

    # Submit a message to the bus for each target language
    to_langs = os.environ['TO_LANG'].split(',')
    for target_lang in to_langs:
        topic_name = os.environ['TRANSLATE_TOPIC']
        if src_lang == target_lang or src_lang == 'und':
            topic_name = os.environ['RESULT_TOPIC']
        message = {
            'text': text,
            'filename': filename,
            'lang': target_lang,
            'src_lang': src_lang
        }
        message_data = json.dumps(message).encode('utf-8')
        topic_path = publisher.topic_path(project_id, topic_name)
        future = publisher.publish(topic_path, data=message_data)
        futures.append(future)
    for future in futures:
        future.result()
    
    
    
    
    
    '''texts = response.text_annotations
    print('Texts:')
    print(texts)
    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))'''
    '''
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))'''
    return texts

In [ ]:
# Call the function to extract text from jpeg image
test = detect_text("test_pdf_ocr/22453181_0_.jpg", credentials)